# Importaciones

In [ ]:
import pandas as pd
import xlrd

# ETL

## Carga de datos y unión

In [ ]:
# Leer los archivos Excel y añadir una columna 'fecha' con la fecha correspondiente al primer día del mes
data_enero = pd.read_excel(r'Datalake\enero.xls')
data_enero['fecha'] = pd.to_datetime('2023-01-01')

data_febrero = pd.read_excel(r'Datalake\febrero.xls')
data_febrero['fecha'] = pd.to_datetime('2023-02-01')

data_marzo = pd.read_excel(r'Datalake\marzo.xls')
data_marzo['fecha'] = pd.to_datetime('2023-03-01')

data_abril = pd.read_excel(r'Datalake\abril.xls')
data_abril['fecha'] = pd.to_datetime('2023-04-01')

data_mayo = pd.read_excel(r'Datalake\mayo.xls')
data_mayo['fecha'] = pd.to_datetime('2023-05-01')

In [ ]:
# Combinar todos los dataframes en uno solo
df_data = pd.concat([data_enero, data_febrero, data_marzo, data_abril, data_mayo], ignore_index=True)

In [ ]:
# Seleccionamos las columnas de interés
df_data = df_data[['codigo', 'nombre', 'unidades', 'venta', 'fecha']]

# Convertir la columna 'fecha' a tipo datetime y crear una columna 'mes'
df_data['fecha'] = pd.to_datetime(df_data['fecha'])
df_data['mes'] = df_data['fecha'].dt.month_name()

In [ ]:
# Mostrar información general del DataFrame y las primeras filas
print(df_data.info())
df_data.head()

## Correcciones de nombres

In [ ]:
# Corregir mayúsculas y minúsculas en la columna 'nombre'
df_data['nombre'] = df_data['nombre'].str.capitalize()

# Usar replace() para eliminar los puntos en la columna 'nombre'
df_data['nombre'] = df_data['nombre'].str.replace('.', '')

In [ ]:
# Reemplazar nombres específicos para normalización
df_data['nombre'] = df_data['nombre'].str.replace('Pesto brie cherry ch', 'Pesto cherry brie ch')
df_data['nombre'] = df_data['nombre'].str.replace('Salchi a caballo mitad', 'Salchichas a caballo ch')

In [ ]:
# Filtrar y eliminar registros con nombres específicos que no queremos
df_data = df_data[df_data['nombre'] != 'A caballo']
df_data = df_data[df_data['nombre'] != 'Piza mitad y mitad']
df_data = df_data[df_data['nombre'] != 'Promo 5 precocidas']

## Pizzas "Normales" y "Especiales"

In [ ]:
# Lista de nombres de interés para pizzas especiales
nombres_interes = [
    'Degustacion de pizzetas', 'Tacc especial', 'Tacc fugazzeta', 'Tacc hace tu pizza', 'Tacc jamon',
    'Tacc marga napo', 'Tacc margarita', 'Tacc muzzarella', 'Tacc napo jamon', 'Tacc napolitana',
    'Tacc rucula', 'Vegana hace tu pizza', 'Vegana pizza especial', 'Vegana pizza muzza'
]

# Filtrar el DataFrame por los nombres de interés para separar las pizzas especiales
df_especiales = df_data[df_data['nombre'].isin(nombres_interes)]

# Filtrar el DataFrame para obtener las pizzas normales
df_normales = df_data[~df_data['nombre'].isin(nombres_interes)]

In [ ]:
# Función para combinar registros de pizzas normales
def combinar_registros(df):
    # Extraer el nombre base de la pizza
    df['nombre_pizza'] = df['nombre'].str[:-3]  # Eliminar los últimos 3 caracteres (incluyendo el espacio antes de 'gr' o 'ch')
    
    # Crear listas vacías para almacenar los datos combinados
    registros_combinados = []

    # Iterar sobre cada nombre base de pizza único
    for nombre_pizza in df['nombre_pizza'].unique():
        # Filtrar registros para el nombre base de la pizza actual
        df_pizza = df[df['nombre_pizza'] == nombre_pizza]
        
        # Iterar sobre cada mes único
        for mes in df_pizza['mes'].unique():
            # Filtrar registros para el nombre base de la pizza y mes actual
            df_pizza_mes = df_pizza[df_pizza['mes'] == mes]
            
            # Encontrar las unidades y ventas de 'gr' y 'ch' para el mes actual
            unidades_gr = df_pizza_mes[df_pizza_mes['nombre'].str.endswith('gr')]['unidades'].sum()
            unidades_ch = df_pizza_mes[df_pizza_mes['nombre'].str.endswith('ch')]['unidades'].sum()
            venta_gr = df_pizza_mes[df_pizza_mes['nombre'].str.endswith('gr')]['venta'].sum()
            venta_ch = df_pizza_mes[df_pizza_mes['nombre'].str.endswith('ch')]['venta'].sum()

            # Calcular las unidades y ventas totales
            unidades_total = unidades_gr + unidades_ch
            venta_total = venta_gr + venta_ch
            
            # Tomar solo un registro como base, ya que deberían ser iguales para la misma pizza y mes
            base_registro = df_pizza_mes.iloc[0]
            
            # Crear un diccionario con la información combinada por mes
            registro_combinado = {
                'codigo': base_registro['codigo'],
                'nombre': nombre_pizza,
                'fecha': base_registro['fecha'],
                'mes': mes,
                'unidades_gr': unidades_gr,
                'unidades_ch': unidades_ch,
                'venta_gr': venta_gr,
                'venta_ch': venta_ch,
                'unidades_total': unidades_total,
                'venta_total': venta_total
            }
            
            # Agregar el registro combinado a la lista
            registros_combinados.append(registro_combinado)
    
    # Convertir la lista de registros combinados en un DataFrame
    df_combinado = pd.DataFrame(registros_combinados)
    
    return df_combinado

In [ ]:
# Ejemplo de uso de la función para combinar registros de pizzas normales
df_normales = combinar_registros(df_normales)

## Carga al Datawarehouse

In [ ]:
# Guardar los DataFrames resultantes en archivos CSV
df_normales.to_csv(r'Datawarehouse\pizzas_normales.csv', index=False)
df_especiales.to_csv(r'Datawarehouse\pizzas_especiales.csv', index=False)